In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [3]:
#readind and preparing data ( text , label )
import os, sys
import pandas as pd
import numpy as np
df=pd.read_excel('incident13type.xlsx', header=None) ## read le file
df.columns=['tags','post']
print(df[1:5])

   tags                                               post
1    11  عند التاسعة من مساء يوم 29/9/2018 وفي محلة الط...
2     4  اقدم المدعو وليد عدنان مرعي على شتم و اهانة ال...
3    12  أقدم المدعو ناصر رضا على اصدار شك لصالح حسن اح...
4     1  الساعة 7.30 من تاريخه وفي محلة فردان شارع كرام...


In [4]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

In [11]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled

In [12]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.post, df.tags, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [13]:

all_data[:2]

[TaggedDocument(words=['الساعة', '14:45', 'من', 'تاريخه', 'اصطدمت', 'الدراجة', 'النارية', 'نوع', 'لين', 'هاي', 'لون', 'موف', 'صنع', '2017', 'رقم', '874626/م', 'بقيادة', 'المدعو', 'محمد', 'احمد', 'عواضة', 'والدته', 'حنان', 'تولد', '1990', 'لبناني', 'بفان', 'نوع', 'مازدا', 'صنع', '1996', 'لون', 'ابيض', 'رقم', '374181/م', 'بقيادة', 'علي', 'عباس', 'زعيتر', 'والدته', 'نور', 'تولد', '1997', 'لبناني', 'نتج', 'عن', 'الحادث', 'اصابة', 'الاول', 'بكسور', 'في', 'الوجه', 'نقل', 'الى', 'مستشفى', 'الجامعة', 'الاميريكية', 'للمعالجة', '،', 'نظم', 'محضر', 'رقم', '302/248'], tags=['Train_0']),
 TaggedDocument(words=['سلمتنا', 'دورية', 'من', 'فصيلتنا', 'المدعو', 'خالد', 'عيسى', '/', 'سوري', 'بجرم', 'دخول', 'البلاد', 'خلسة', 'وذلك', 'بعد', 'توقيفه', 'في', 'محلة', 'الروشة', 'الساعة', '18:00'], tags=['Train_1'])]

In [14]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

C:\Users\Maya\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
100%|█████████████████████████████████████████████████████████████████████████| 2384/2384 [00:00<00:00, 1192370.71it/s]


In [15]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|█████████████████████████████████████████████████████████████████████████| 2384/2384 [00:00<00:00, 1200819.11it/s]


In [16]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [17]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [19]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)

C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=1, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [20]:
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)

C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [23]:
from sklearn.metrics import accuracy_score, classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7625698324022346
              precision    recall  f1-score   support

           1       0.67      0.77      0.72        83
           2       0.41      0.42      0.41        31
           3       0.93      0.93      0.93       120
           4       0.31      0.24      0.27        21
           5       0.44      0.53      0.48        15
           6       0.31      0.25      0.28        20
           7       0.89      0.82      0.85       150
           8       0.79      0.73      0.76        15
           9       0.72      0.87      0.79        62
          10       0.56      0.49      0.52        39
          11       0.95      0.90      0.93        63
          12       0.89      1.00      0.94        25
          13       0.72      0.69      0.71        72

   micro avg       0.76      0.76      0.76       716
   macro avg       0.66      0.67      0.66       716
weighted avg       0.76      0.76      0.76       716

